In [4]:
import getpass
import os

os.environ["ANTHROPIC_API_KEY"] = getpass.getpass()

In [5]:
import anthropic

client = anthropic.Anthropic()

client.models.list(limit=20)

SyncPage[ModelInfo](data=[ModelInfo(id='claude-3-5-sonnet-20241022', created_at=datetime.datetime(2024, 10, 22, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude 3.5 Sonnet (New)', type='model'), ModelInfo(id='claude-3-5-haiku-20241022', created_at=datetime.datetime(2024, 10, 22, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude 3.5 Haiku', type='model'), ModelInfo(id='claude-3-5-sonnet-20240620', created_at=datetime.datetime(2024, 6, 20, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude 3.5 Sonnet (Old)', type='model'), ModelInfo(id='claude-3-haiku-20240307', created_at=datetime.datetime(2024, 3, 7, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude 3 Haiku', type='model'), ModelInfo(id='claude-3-opus-20240229', created_at=datetime.datetime(2024, 2, 29, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude 3 Opus', type='model'), ModelInfo(id='claude-3-sonnet-20240229', created_at=datetime.datetime(2024, 2, 29, 0, 0, tzinfo=datetime.timezone.utc), di

In [4]:
our_first_message = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1000,
    messages=[
        {"role": "user", "content": "Hi there! Please write me a haiku about an orca whale."}
    ]
)

print(our_first_message.content[0].text)

Here's a haiku about an orca whale:

Killer whale glides by,
Powerful, majestic grace,
Ruler of the deep.


In [ ]:
message = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1000,
    temperature=0,
    system="You are a world-class poet. Respond only with short poems.",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Why is the ocean salty?"
                }
            ]
        }
    ]
)
print(message.content)

In [5]:
def translate(word, language):
  response = client.messages.create(
      model="claude-3-haiku-20240307",
      max_tokens=1000,
      messages=[
          {"role": "user", "content": f"Translate the word {word} into {language}.  Only respond with the translated word, nothing else"}
      ]
  )
  return response.content[0].text

In [6]:
translate("rainbow", "German")

'Regenbogen'

In [7]:
conversation_history = []

while True:
    user_input = input("User: ")
    
    if user_input.lower() == "quit":
        print("Conversation ended.")
        break
    
    conversation_history.append({"role": "user", "content": user_input})

    response = client.messages.create(
        model="claude-3-haiku-20240307",
        messages=conversation_history,
        max_tokens=500
    )

    assistant_response = response.content[0].text
    print(f"Assistant: {assistant_response}")
    conversation_history.append({"role": "assistant", "content": assistant_response})

Assistant: Getting rejected for a job can be really disappointing and discouraging. Here are some tips to help you get in the right mindset after facing a few rejections:

1. Remember it's not personal: Job rejections are rarely a reflection on your worth or capabilities. There are often many qualified candidates applying for the same role, and the decision often comes down to factors outside your control.

2. Reframe the experience: Instead of seeing the rejections as failures, try to view them as opportunities to learn and grow. Reflect on the feedback you received and think about how you can improve for the next application.

3. Focus on what you can control: You can't control whether you get a job offer, but you can control your mindset, the effort you put into your search, and your ability to learn and grow from each experience.

4. Celebrate small wins: Acknowledge and celebrate the progress you're making, whether it's landing an interview, getting positive feedback, or simply ta

BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'messages.2: all messages must have non-empty content except for the optional final assistant message'}}

In [8]:
stream = client.messages.create(
    messages=[
        {
            "role": "user",
            "content": "How to lead a conversation?",
        }
    ],
    model="claude-3-haiku-20240307",
    max_tokens=1000,
    temperature=0,
    stream=True,
)
for event in stream:
    if event.type == "message_start":
        input_tokens = event.message.usage.input_tokens
        print("MESSAGE START EVENT", flush=True)
        print(f"Input tokens used: {input_tokens}", flush=True)
        print("========================")
    elif event.type == "content_block_delta":
        print(event.delta.text, flush=True, end="")
    elif event.type == "message_delta":
        output_tokens = event.usage.output_tokens
        print("\n========================", flush=True)
        print("MESSAGE DELTA EVENT", flush=True)
        print(f"Output tokens used: {output_tokens}", flush=True)
        

MESSAGE START EVENT
Input tokens used: 13
Here are some tips for leading a conversation effectively:

1. Start with open-ended questions. Ask questions that can't be answered with just a yes or no, like "How was your weekend?" or "What have you been up to lately?" This encourages the other person to share more.

2. Listen actively. Pay attention to what the other person is saying and ask follow-up questions to show you're engaged. Avoid interrupting them.

3. Find common ground. Look for shared interests, experiences or perspectives that you can build the conversation around.

4. Share relevant information about yourself. Offer insights or anecdotes that relate to the topic, but don't dominate the conversation.

5. Guide the flow. Gently steer the conversation in a direction that interests you both. You can introduce new topics or themes when the current one starts to wind down.

6. Be positive and enthusiastic. Use an upbeat tone and body language to create an engaging atmosphere.

7.

In [ ]:
from anthropic import AsyncAnthropic

client = AsyncAnthropic()

async def streaming_with_helpers():
    async with client.messages.stream(
        max_tokens=1024,
        messages=[
            {
                "role": "user",
                "content": "Write me sonnet about orchids",
            }
        ],
        model="claude-3-haiku-20240307",
    ) as stream:
        async for text in stream.text_stream:
            print(text, end="", flush=True)

    final_message = await stream.get_final_message()
    print("\n\nSTREAMING IS DONE.  HERE IS THE FINAL ACCUMULATED MESSAGE: ")
    print(final_message.to_json())

await streaming_with_helpers()

# Image Prompting

In [1]:
import base64
import mimetypes

def create_image_message(image_path):
    # Open the image file in "read binary" mode
    with open(image_path, "rb") as image_file:
        # Read the contents of the image as a bytes object
        binary_data = image_file.read()
    
    # Encode the binary data using Base64 encoding
    base64_encoded_data = base64.b64encode(binary_data)
    
    # Decode base64_encoded_data from bytes to a string
    base64_string = base64_encoded_data.decode('utf-8')
    
    # Get the MIME type of the image based on its file extension
    mime_type, _ = mimetypes.guess_type(image_path)
    
    # Create the image block
    image_block = {
        "type": "image",
        "source": {
            "type": "base64",
            "media_type": mime_type,
            "data": base64_string
        }
    }
    
    
    return image_block

In [6]:
messages = [
    {
        "role": "user",
        "content": [
            create_image_message("./IMG_5381.PNG")
        ]
    }
]

response = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1000,
    messages=messages
)
print(response.content[0].text)

The image depicts a serene and picturesque natural landscape. It shows a person standing on a rocky outcrop, overlooking a lush, forested valley with a turquoise-colored lake in the distance. The surrounding environment is characterized by towering evergreen trees, mossy rocks, and a clear blue sky. The person appears to be taking in the breathtaking view, seemingly enjoying the tranquility and beauty of this wilderness setting.


In [7]:
import base64
import httpx

image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Church_of_light.jpg/1599px-Church_of_light.jpg"
image_media_type = "image/jpeg"
image_data = base64.b64encode(httpx.get(image_url).content).decode("utf-8")

messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": image_media_type,
                        "data": image_data,
                    },
                },
                {
                    "type": "text",
                    "text": "Describe this image."
                }
            ],
        }
    ]


response = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1000,
    messages=messages
)
print(response.content[0].text)



This image depicts a serene and picturesque winter landscape in Iceland. In the foreground, there is a small church with a distinctive red-roofed steeple, surrounded by a snowy, rocky landscape. In the sky above, the stunning natural phenomenon of the Northern Lights, or aurora borealis, can be seen in vibrant green and white shimmering patterns. The church stands alone against the backdrop of the rugged mountainous terrain, creating a striking contrast between the man-made structure and the untamed natural environment. The overall scene conveys a sense of remote, wintry beauty and the awe-inspiring power of nature.


In [ ]:
import base64
import mimetypes

research_paper_pages = [
    "./images/research_paper/page1.png",
    "./images/research_paper/page2.png",
    "./images/research_paper/page3.png",
    "./images/research_paper/page4.png",
    "./images/research_paper/page5.png"
    ]

def create_image_message(image_path):
    # Open the image file in "read binary" mode
    with open(image_path, "rb") as image_file:
        # Read the contents of the image as a bytes object
        binary_data = image_file.read()
    
    # Encode the binary data using Base64 encoding
    base64_encoded_data = base64.b64encode(binary_data)
    
    # Decode base64_encoded_data from bytes to a string
    base64_string = base64_encoded_data.decode('utf-8')
    
    # Get the MIME type of the image based on its file extension
    mime_type, _ = mimetypes.guess_type(image_path)
    
    # Create the image block
    image_block = {
        "type": "image",
        "source": {
            "type": "base64",
            "media_type": mime_type,
            "data": base64_string
        }
    }
    
    
    return image_block

def transcribe_single_page(page_url):
    messages = [
    {
        "role": "user",
        "content": [
            create_image_message(page_url),
            {"type": "text", "text": "transcribe the text from this page of a research paper as accurately as possible."}
        ]
    }
    ]

    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=5000,
        messages=messages
    )
    return response.content[0].text

def summarize_paper(pages):
    complete_paper_text = ""
    for page in pages:
        print("transcribing page ", page)
        transribed_text = transcribe_single_page(page)
        print(transribed_text[:200])
        complete_paper_text += transribed_text
    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=5000,
        messages=[
            {
                "role": "user",
                "content": f"This is the transcribed contents of a research paper <paper>{complete_paper_text}</paper>.  Please summarize this paper for a non-research audience in at least 3 paragraphs.  Make to sure explain any abbreviations or technical jargon, and use analogies when possible"
            }
        ]
    )
    print(response.content[0].text)
